In [1]:
# Phân loại domain sử dụng XGBoost
!free

              total        used        free      shared  buff/cache   available
Mem:      164620976    38465772    19144524      165908   107010680   124942000
Swap:      16777212        4616    16772596


In [2]:
!pwd

/home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict


In [3]:
import sklearn
import pandas as pd
from io import open as open_unicode
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from collections import Counter
from xgboost import XGBClassifier
import time
import os

/home/polynote/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# PARAMS

In [4]:
# input - PARAMS
# root_dir = '/home/datamon/anhdt157/'
current_root = '/home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/'

In [5]:
# output path
MODE = 'vi' # NEED TO MODIFY vi or en
MONTH = '202001' # NEED TO MODIFY
TREE_VERSION = '8' #MODIFY - FOR UNIFING

In [6]:
input_test_cant_parsing = current_root + 'train_test_data/test/dinfo-data-test-cant-parsing-' + MONTH + '.csv'
input_test_spam = current_root + 'train_test_data/test/data-test-spam-' + MONTH + '.csv'
# dict IID to label
input_dict_IID_use = current_root + 'train_test_data/dict_IID_label/' + MODE + '-dict-IID-label-' + MONTH + '.csv'

In [7]:
model_path = current_root + 'model/model-each-month/' + MODE + '-model-' + MONTH + '.pkl'
vector_path = current_root + 'model/model-each-month/' + MODE + '-vectorizer-' + MONTH + '.pkl'

In [8]:
OUTPUT_PREDICT_ONLY_PREDICT = current_root + 'result_predict/' + MODE + '_predict' + '_' + MONTH + '_only_predict.csv'

In [9]:
PATH_EXCEPT_TREE = current_root + 'old_logic/tree/Except_keywords_ml_v8_20200220.xlsx'

In [10]:
df_test_spam = pd.read_csv(input_test_spam, "|")
df_test_cant_parsing = pd.read_csv(input_test_cant_parsing, "|")

In [11]:
len(df_test_spam)

892021

In [12]:
df_test_spam.head(2)

,domain_id,domain,IID,level_1,level_2,level_3,time,date,tree_version,lang,title_desc_preprocess
0,516,www.vinapas.com,A029999,A000000,A020000,A029900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
1,516,www.vinapas.com,E999999,E000000,E990000,E999900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...


In [13]:
# return dict_IID_to_label and dict_label_to_IID
def read_dict_IID_label_file(path_dict):
    df_IID_to_label = pd.read_csv(path_dict, delimiter='|')
    dict_label_to_IID = {}
    dict_IID_to_label = {}
    for index, row in df_IID_to_label.iterrows():
        dict_IID_to_label[row['IID']] = row['label']
        dict_label_to_IID[row['label']] = row['IID']
    return dict_label_to_IID, dict_IID_to_label

In [14]:
dict_label_to_IID_use, dict_IID_to_label_use = read_dict_IID_label_file(input_dict_IID_use)

In [15]:
df_test_spam_use = df_test_spam[df_test_spam['lang'] == MODE]
df_test_cant_parsing_use = df_test_cant_parsing[df_test_cant_parsing['lang'] == MODE]

In [16]:
len(df_test_spam_use)

333551

In [17]:
df_test_spam_use.head(2)

,domain_id,domain,IID,level_1,level_2,level_3,time,date,tree_version,lang,title_desc_preprocess
0,516,www.vinapas.com,A029999,A000000,A020000,A029900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
1,516,www.vinapas.com,E999999,E000000,E990000,E999900,1582598448,202001,8,vi,trang_chủ_công_ty thhh thương_mại quốc_tế vina...


In [18]:
df_test_cant_parsing_use.head(2)

,domain_clean,title_desc_preprocess,lang


In [19]:
df_test_cant_parsing_use = df_test_cant_parsing_use.drop(['lang'], axis=1)
df_test_cant_parsing_use = df_test_cant_parsing_use.rename(columns={"domain_clean": "domain"})
df_test_cant_parsing_use = df_test_cant_parsing_use.drop_duplicates(subset=['domain'])

In [20]:
df_test_cant_parsing_use.head(5)

,domain,title_desc_preprocess


In [21]:
df_test_spam_use = df_test_spam_use.drop(['domain_id', 'IID', 'level_1', 'level_2', 'level_3', 'time', 'date', 'tree_version', 'lang'], axis=1)
df_test_spam_use = df_test_spam_use.drop_duplicates(subset=['domain'])

In [22]:
len(df_test_spam_use)

150250

In [23]:
df_test_spam_use.head(5)

,domain,title_desc_preprocess
0,www.vinapas.com,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
3,baobithiennienky.com,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...
6,intuipe.com,công_ty_tnhh sx tm tổng_hợp ii sản_xuất in bao...
9,baobithuanphat.bizz.vn,công_ty_tnhh đầu_tư bao_bì thuận_phát bao_bì t...
12,trongdung.vn,cửa_hàng tạp_hóa_trọng dũng sản_phẩm_giá tốt l...


In [24]:
df_need_predict = df_test_spam_use.append(df_test_cant_parsing_use, ignore_index=True)

In [25]:
df_need_predict.head(5)

,domain,title_desc_preprocess
0,www.vinapas.com,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
1,baobithiennienky.com,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...
2,intuipe.com,công_ty_tnhh sx tm tổng_hợp ii sản_xuất in bao...
3,baobithuanphat.bizz.vn,công_ty_tnhh đầu_tư bao_bì thuận_phát bao_bì t...
4,trongdung.vn,cửa_hàng tạp_hóa_trọng dũng sản_phẩm_giá tốt l...


In [26]:
len(df_need_predict)

150250

### DF_EXCEPT

In [27]:
df = pd.read_excel(PATH_EXCEPT_TREE)
df.head(1)

,IID,Review result,title-IID,Prob,threshold_prob,total_domains,remain_domains (be not classified),keywords_except,Problem
0,A010199,OK,"Nhà in ấn, xuất bản địa phương",low + high,0.1,2140,0,NaN,NaN


In [28]:
df

,IID,Review result,title-IID,Prob,threshold_prob,total_domains,remain_domains (be not classified),keywords_except,Problem
0,A010199,OK,"Nhà in ấn, xuất bản địa phương",low + high,0.10,2140,0,NaN,NaN
1,A010299,OK,"Quảng cáo online, website",low + high,0.10,3736,0,NaN,NaN
2,A020199,NOT OK with low prob,Nông nghiệp,low + high,0.19,191198,188373,NaN,NaN
3,A020299,OK,Lâm nghiệp,high,0.10,209,0,NaN,NaN
4,A020399,OK,Ngư nghiệp & Thủy sản,high,0.10,434,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
108,H010399,OK,Phần mềm,low + high,0.10,7805,0,"camera, nha",NaN
109,H020199,OK,Thiết bị nghe,low + high,0.10,1024,0,NaN,NaN
110,H020299,OK,Máy ảnh,low + high,0.10,866,0,NaN,NaN
111,H020399,OK,Thiết bị & đồ điện tử gia dụng,low + high,0.10,1863,0,"trực_tuyến, online, apk",NaN


In [29]:
def generate_list_des_domain(dfx, type_):
    dict_tokenized_vi = {}
    keys = []
    for index, row in dfx.iterrows():
        IID = row[type_]
        title_desc = row['title_desc_preprocess']
        if IID not in keys:
            keys.append(IID)
            new_child_list = []
            new_child_list.append(title_desc)
            #ben duoi chu y check co trong list chua
            dict_tokenized_vi[IID] = new_child_list
        else:
            old_child_list = dict_tokenized_vi[IID]
            old_child_list.append(title_desc)
    print('length-df = ', len(dfx))
    return dict_tokenized_vi

In [30]:
dict_IID_threshold = {}
dict_IID_keywords_except = {}
keys = []
for row in df.itertuples():
    IID = row.IID
    string_execpt = str(row.keywords_except)
    threshold = row.threshold_prob
    if IID not in keys:
        keys.append(IID)
        dict_IID_threshold[IID] = threshold
        if string_execpt != 'nan':
            dict_IID_keywords_except[IID] = string_execpt.split(', ')
        else:
            list_keywords = []
            dict_IID_keywords_except[IID] = list_keywords
# print(dict_IID_threshold) 
# print('============')
# print(dict_IID_keywords_except) 

In [31]:
len(dict_IID_keywords_except['A010199'])

0

# PREDICT

In [32]:
df_need_predict.head(5)

,domain,title_desc_preprocess
0,www.vinapas.com,trang_chủ_công_ty thhh thương_mại quốc_tế vina...
1,baobithiennienky.com,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...
2,intuipe.com,công_ty_tnhh sx tm tổng_hợp ii sản_xuất in bao...
3,baobithuanphat.bizz.vn,công_ty_tnhh đầu_tư bao_bì thuận_phát bao_bì t...
4,trongdung.vn,cửa_hàng tạp_hóa_trọng dũng sản_phẩm_giá tốt l...


In [33]:
# create new folder
def util_mkdir(dir):
    if not os.path.exists(dir):
        os.mkdir(dir)

In [34]:
def keep_top_value_in_dict(input_dict, number=5):
    return {k: v for k, v in sorted(list(input_dict.items()), key=lambda x: x[1], reverse=True)[:number]}

In [35]:
dict_index_with_domain_clean_use = {}
count_x = 0
for index, row in df_need_predict.iterrows():
    dict_index_with_domain_clean_use[count_x] = row['domain']
    count_x += 1

In [36]:
def have_except_keywords(list_except, title_desc):
    list_title_desc = title_desc.split()
    for word in list_title_desc:
        if word in list_except:
            return True
    return False

In [37]:
# Return list IID-label
def get_list_IID_label(dict_str, title_desc):
    list_label_result = []
    list_IID_result = []
    list_prob = []
    
    max_value = dict_str[list(dict_str)[0]]
    label_max_value = list(dict_str)[0]
    
    if max_value < 0.1:
        list_label_result.append(label_max_value)
        list_IID_result.append(dict_label_to_IID_use[label_max_value])
        list_prob.append(max_value)
    else:
        for label, prob in dict_str.items():
            if len(list_label_result) >= 3:
                break
            IID = dict_label_to_IID_use[label]
            
            threshold = dict_IID_threshold[IID]
            list_keywords = dict_IID_keywords_except[IID] # Can loai di??
            
            if prob >= threshold and not have_except_keywords(list_keywords, title_desc):
                list_label_result.append(label)
                list_IID_result.append(dict_label_to_IID_use[label])
                list_prob.append(prob)
#             elif have_except_keywords(list_keywords, title_desc):
#                 print('IID = ', IID)
#                 print('prob = ', prob)
#                 print('threshold = ', threshold)
#                 print('list_keywords = ', str(list_keywords))
#                 print('title_desc = ', str(title_desc))
#                 print('=========================')
    return list_label_result, list_IID_result, list_prob

class Classification:
    def __init__(self):
        self.model = None
        self.vectorizer = None
        
    def load(self, model):
        print('loading %s ...' % (model))
        if os.path.isfile(model):
            return joblib.load(model)
        else:
            return None
    def load_model(self):
        # NEED TO MODIFY
        self.vectorizer = self.load(vector_path)
        self.model = self.load(model_path)
        
    def evaluation(self, x):
        y_pred = self.model.predict(x)
        y_prob = self.model.predict_proba(x)
        return y_pred, y_prob
    
    def feature_extraction(self, X):
#         X = X.values.astype('U')
        if self.vectorizer == None:
            self.vectorizer = TfidfVectorizer()
            self.vectorizer.fit(X)
            self.vectorizer.stop_words_ = None
        return self.vectorizer.transform(X)
    
    def test(self):
        self.load_model()
        x_test = self.prepare_data(df_need_predict) # Chuyển sang list các domain để predict
        x_test_vec = self.feature_extraction(x_test)
        y_pred, y_prob = self.evaluation(x_test_vec)
        # write file
        self.process_and_save_prob_data(y_pred, y_prob, 
                                        OUTPUT_PREDICT_ONLY_PREDICT,
                                       x_test)
#         self.save_model()
    def process_and_save_prob_data(self, y_pre, y_prob, output_path, x_title_desc):
        # example: [{0: 0.98, 1: 0.66, 2: 0.42, 3: 0.64, 4: 0.12, 5: 0.26}, {0: 0.98, 1: 0.66, 2: 0.42, 3: 0.64, 4: 0.12, 5: 0.26},...]
        list_each_dict = []
        for i in range(len(y_prob)):
            dict_prob = {}
            # Với từng dòng là list các xác suất tương ứng, i là index của mỗi dòng, tương ứng thứ tự của domain
            for j in range(len(y_prob[i])):
                dict_prob[j] = y_prob[i][j]
            dict_prob = keep_top_value_in_dict(dict_prob)
            list_each_dict.append(dict_prob)
        # Save data
        print("Writing data to csv file", output_path)
        with open(output_path, "w") as f:
            f_writer = csv.writer(f, delimiter='|')
            f_writer.writerow(
                ['domain_clean', 'IID', 'title_desc_preprocess', 'date', 'tree_version', 'lang', 'prob'])
            for i, value in enumerate(y_pre):
                # write each domain ----'
                label_y_predict, IID_y_predict, list_prob = get_list_IID_label(list_each_dict[i], x_title_desc[i])
                for i_, IID_ in enumerate(IID_y_predict):
                    f_writer.writerow(
                        [dict_index_with_domain_clean_use[i],
                         IID_,
                         x_title_desc[i], MONTH, TREE_VERSION, MODE, list_prob[i_]])
        print("Done writing csv data")

    def prepare_data(self, df_dataset):
        # Trả về list các domain để dự đoán
        x = []
        for name, row in df_dataset.iterrows():
            x.append(row['title_desc_preprocess'])
        return x
    
    def run_test(self):
        self.test()
        
    # predict cho từng domain để test
    def main_predict(self, domain_title_desc):
        self.load_model()
        x_test = [domain_title_desc]
        x_test_vec = self.feature_extraction(x_test)
        y_pred, y_prob = self.evaluation(x_test_vec)
        print('y_pred', y_pred)
        print('y_prob', y_prob)

In [38]:
c = Classification()
c.run_test()

loading /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/model/model-each-month/vi-vectorizer-202001.pkl ...
loading /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/model/model-each-month/vi-model-202001.pkl ...
Writing data to csv file /home/polynote/notebooks/anhdt157/classify-use-ml/end-to-end-final-predict/result_predict/vi_predict_202001_only_predict.csv
Done writing csv data


In [39]:
#====READT TEST FILE - ONLY PREDICT=====
df_predict_result = pd.read_csv(OUTPUT_PREDICT_ONLY_PREDICT, delimiter='|')

In [40]:
len(df_predict_result)

123309

In [41]:
df_predict_result.head(5)

,domain_clean,IID,title_desc_preprocess,date,tree_version,lang,prob
0,www.vinapas.com,A020199,trang_chủ_công_ty thhh thương_mại quốc_tế vina...,202001,8,vi,0.988618
1,baobithiennienky.com,A020199,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...,202001,8,vi,0.992758
2,intuipe.com,A020199,công_ty_tnhh sx tm tổng_hợp ii sản_xuất in bao...,202001,8,vi,0.999828
3,baobithuanphat.bizz.vn,A020199,công_ty_tnhh đầu_tư bao_bì thuận_phát bao_bì t...,202001,8,vi,0.999986
4,trongdung.vn,A020199,cửa_hàng tạp_hóa_trọng dũng sản_phẩm_giá tốt l...,202001,8,vi,0.870306


In [42]:
df_predict_result = df_predict_result.rename(columns={"domain_clean": "domain"})

In [43]:
df_predict_result.head(5)

,domain,IID,title_desc_preprocess,date,tree_version,lang,prob
0,www.vinapas.com,A020199,trang_chủ_công_ty thhh thương_mại quốc_tế vina...,202001,8,vi,0.988618
1,baobithiennienky.com,A020199,công_ty_tnhh thiên_niên_kỷ tuấn anh chuyên sản...,202001,8,vi,0.992758
2,intuipe.com,A020199,công_ty_tnhh sx tm tổng_hợp ii sản_xuất in bao...,202001,8,vi,0.999828
3,baobithuanphat.bizz.vn,A020199,công_ty_tnhh đầu_tư bao_bì thuận_phát bao_bì t...,202001,8,vi,0.999986
4,trongdung.vn,A020199,cửa_hàng tạp_hóa_trọng dũng sản_phẩm_giá tốt l...,202001,8,vi,0.870306


# END